In [8]:
import os
import glob
import pandas as pd

In [2]:
path = os.path.join(os.pardir, "datasets","5G-production-dataset")


In [9]:
files = glob.glob(path + "/**/*.csv", recursive=True)

In [10]:
listdf = []
for i, file in enumerate(files):
  file = file.replace("\\","/")
  df = pd.read_csv(file)
  folders_name = file.split('/')[3:-1]

  # It will identify if the user is static or moving.
  df["Mobility_Pattern"] = folders_name[1]
  folders_name.pop(1)

  # It will identify if the user is downloading or watching some streaming.
  df["User_Activity"] = ' '.join(folders_name)

  # It will identify if the user is downloading or watching some streaming.
  df["Log_Number"] = i

  listdf.append(df)

del df

In [11]:
df = pd.concat(listdf, axis=0, ignore_index=True)

In [12]:
df

,Timestamp,Longitude,Latitude,Speed,Operatorname,CellID,NetworkMode,RSRP,RSRQ,SNR,...,PINGLOSS,CELLHEX,NODEHEX,LACHEX,RAWCELLID,NRxRSRP,NRxRSRQ,Mobility_Pattern,User_Activity,Log_Number
0,2019.11.28_07.27.57,-8.388193,51.935608,0,B,12,5G,-102,-10,8.0,...,-,C,A81B,9CBA,11016972,-102.0,-1.0,Driving,Amazon_Prime animated-AdventureTime,0
1,2019.11.28_07.27.57,-8.388269,51.935542,1,B,12,5G,-102,-10,8.0,...,-,C,A81B,9CBA,11016972,-102.0,-1.0,Driving,Amazon_Prime animated-AdventureTime,0
2,2019.11.28_07.27.58,-8.388269,51.935542,1,B,12,5G,-102,-10,8.0,...,-,C,A81B,9CBA,11016972,-102.0,-1.0,Driving,Amazon_Prime animated-AdventureTime,0
3,2019.11.28_07.27.59,-8.388269,51.935542,1,B,12,5G,-102,-10,3.0,...,-,C,A81B,9CBA,11016972,-101.0,-3.0,Driving,Amazon_Prime animated-AdventureTime,0
4,2019.11.28_07.28.00,-8.388269,51.935542,1,B,12,5G,-102,-10,3.0,...,-,C,A81B,9CBA,11016972,-101.0,-3.0,Driving,Amazon_Prime animated-AdventureTime,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188706,2019.12.16_13.36.25,-8.394601,51.886139,0,B,11,5G,-101,-13,-7.0,...,-,B,A4DF,9CBA,10805003,-103.0,-20.0,Static,Netflix Season3-StrangerThings,82
188707,2019.12.16_13.36.27,-8.394601,51.886139,0,B,11,5G,-101,-13,-7.0,...,-,B,A4DF,9CBA,10805003,-103.0,-20.0,Static,Netflix Season3-StrangerThings,82
188708,2019.12.16_13.36.28,-8.394601,51.886139,0,B,11,5G,-103,-12,5.0,...,-,B,A4DF,9CBA,10805003,-105.0,-19.0,Static,Netflix Season3-StrangerThings,82
188709,2019.12.16_13.36.29,-8.394601,51.886139,0,B,11,5G,-103,-12,5.0,...,-,B,A4DF,9CBA,10805003,-105.0,-19.0,Static,Netflix Season3-StrangerThings,82
